In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import numpy as np

train_df = pd.read_csv('train_final.csv')
test_df = pd.read_csv('test_final (1).csv')

categorical_columns = train_df.select_dtypes(include=['object']).columns
imputer = SimpleImputer(strategy='most_frequent')

train_df[categorical_columns] = imputer.fit_transform(train_df[categorical_columns])
test_df[categorical_columns] = imputer.transform(test_df[categorical_columns])

label_encoder = LabelEncoder()
combined_data = pd.concat([train_df, test_df])

combined_data[categorical_columns] = combined_data[categorical_columns].astype(str)

for col in categorical_columns:
    combined_data[col] = label_encoder.fit_transform(combined_data[col])

train_df = combined_data.iloc[:train_df.shape[0], :]
test_df = combined_data.iloc[train_df.shape[0]:, :]

X = train_df.drop(columns=['income>50K'])
y = train_df['income>50K']

X = X.drop(columns=['ID'], errors='ignore')
test_df = test_df.drop(columns=['income>50K'], errors='ignore')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_columns = X_train.select_dtypes(include=['int64', 'float64']).columns
imputer_numeric = SimpleImputer(strategy='median')
X_train[numeric_columns] = imputer_numeric.fit_transform(X_train[numeric_columns])
X_val[numeric_columns] = imputer_numeric.transform(X_val[numeric_columns])

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.3],
    'min_child_weight': [1, 3, 5]
}

random_search = RandomizedSearchCV(
    xgb_model, param_distributions=param_grid,
    scoring='roc_auc', n_iter=50, cv=5, random_state=42, verbose=1, n_jobs=-1
)

random_search.fit(X_train, y_train)

best_xgb_model = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)
print("Best AUC Score from tuning:", random_search.best_score_)

y_val_pred_proba = best_xgb_model.predict_proba(X_val)[:, 1]
auc_score = roc_auc_score(y_val, y_val_pred_proba)
print(f"Validation AUC Score after tuning: {auc_score}")

if 'ID' not in test_df.columns:
    test_df['ID'] = range(1, len(test_df) + 1)

test_df[numeric_columns] = imputer_numeric.transform(test_df[numeric_columns])

test_predictions = best_xgb_model.predict_proba(test_df.drop(columns=['ID'], errors='ignore'))[:, 1]

submission = pd.DataFrame({'ID': test_df['ID'], 'Prediction': test_predictions})
submission.to_csv('submission_xgb.csv', index=False)

print("Submission file created: submission_xgb.csv")


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:15:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'subsample': 1.0, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.6}
Best AUC Score from tuning: 0.9278211077879138
Validation AUC Score after tuning: 0.9251774024221552
Submission file created: submission_xgb.csv
